In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import re
import nltk
from nltk.tokenize import word_tokenize

try:
    nltk.download('punkt')
except:
    pass

train_data = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
test_data  = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

X_train, y_train = train_data.data, train_data.target
X_test, y_test = test_data.data, test_data.target

print(f"Liczba dokumentów w zbiorze treningowym: {len(X_train)}")
print(f"Liczba dokumentów w zbiorze testowym: {len(X_test)}")

def clean_and_tokenize(text):
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = word_tokenize(text)
    return tokens

X_train_tokens = [clean_and_tokenize(doc) for doc in X_train]
X_test_tokens = [clean_and_tokenize(doc) for doc in X_test]

print("\nPrzykład tokenów z 1. dokumentu (pierwsze 30 tokenów):")
print(X_train_tokens[0][:30])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rafal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Liczba dokumentów w zbiorze treningowym: 11314
Liczba dokumentów w zbiorze testowym: 7532

Przykład tokenów z 1. dokumentu (pierwsze 30 tokenów):
['i', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'i', 'saw', 'the', 'other', 'day', 'it', 'was', 'a', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late']


## Zadanie domowe


In [2]:
import tensorflow_datasets as tfds # Do ładowania zbioru IMDB
import numpy as np                 # Do ogólnych operacji numerycznych
import re                          # Do wyrażeń regularnych (czyszczenia tekstu)
import nltk                        # Do operacji NLP
from nltk.tokenize import word_tokenize # Do dzielenia tekstu na słowa (tokeny)

# Pobieranie niezbędnego pakietu NLTK do tokenizacji.
# Wykonuje się tylko raz.
try:
    nltk.download('punkt')
except:
    pass

# --- KROK 1: ŁADOWANIE DANYCH IMDB REVIEWS ---

# Ładowanie zbioru danych IMDB Reviews
# split=['train', 'test']: dzieli na zbiór treningowy i testowy
# as_supervised=True: zwraca dane jako krotki (tekst, etykieta)
(ds_train, ds_test), ds_info = tfds.load(
    'imdb_reviews',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# Konwersja zbiorów TensorFlow na listy Pythona
# Użycie tfds.as_numpy() konwertuje dane, a następnie:
# - doc (recenzja) to ciąg bajtów (bytes), który musimy zdekodować do str
# - label (sentyment: 0 lub 1) jest już gotowy
print("--- Ładowanie i konwersja danych ---")

# Zbiór treningowy (tekst i etykiety)
X_train_imdb = [doc.decode('utf8') for doc, label in tfds.as_numpy(ds_train)]
y_train_imdb = [label for doc, label in tfds.as_numpy(ds_train)]

# Zbiór testowy (tekst i etykiety)
X_test_imdb = [doc.decode('utf8') for doc, label in tfds.as_numpy(ds_test)]
y_test_imdb = [label for doc, label in tfds.as_numpy(ds_test)]

print(f"Liczba recenzji treningowych: {len(X_train_imdb)}") # Powinno być 25000
print(f"Liczba recenzji testowych: {len(X_test_imdb)}")     # Powinno być 25000


# --- KROK 2: DEFINICJA FUNKCJI CZYSZCZĄCEJ I TOKENIZUJĄCEJ ---

def clean_and_tokenize(text):
    # 1. Usuń znaki niebędące literami (zachowaj a-z, A-Z i białe znaki)
    # Odpowiada to funkcji clean_text ze zrzutu
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    # 2. Zmień wszystkie litery na małe
    text = text.lower()

    # 3. Usuń nadmiarowe białe znaki (np. podwójne spacje po usunięciu znaków) i przytnij
    text = re.sub(r'\s+', ' ', text).strip()

    # 4. Tokenizacja (podział na pojedyncze słowa/tokeny)
    tokens = word_tokenize(text)

    return tokens


# --- KROK 3: ZASTOSOWANIE CZYSZCZENIA I TOKENIZACJI ---

print("\n--- Rozpoczęcie czyszczenia i tokenizacji ---")

# Zastosowanie funkcji do danych treningowych
X_train_tokens_imdb = [clean_and_tokenize(doc) for doc in X_train_imdb]

# Zastosowanie funkcji do danych testowych
X_test_tokens_imdb = [clean_and_tokenize(doc) for doc in X_test_imdb]


# --- KROK 4: WERYFIKACJA WYNIKÓW ---

print("\n--- Przykład przetworzonego dokumentu ---")
print(f"Oryginalna recenzja (fragment): {X_train_imdb[0][:150]}...")
print(f"Sentyment (0=negatywny, 1=pozytywny): {y_train_imdb[0]}")
print("Tokeny po przetworzeniu (pierwsze 30):")
print(X_train_tokens_imdb[0][:30])

# Teraz zmienne X_train_tokens_imdb i X_test_tokens_imdb zawierają listy list słów (tokenów),
# gotowe do dalszej analizy (np. liczenia częstości słów lub budowy modelu).

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rafal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


--- Ładowanie i konwersja danych ---
Liczba recenzji treningowych: 25000
Liczba recenzji testowych: 25000

--- Rozpoczęcie czyszczenia i tokenizacji ---

--- Przykład przetworzonego dokumentu ---
Oryginalna recenzja (fragment): This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be the...
Sentyment (0=negatywny, 1=pozytywny): 0
Tokeny po przetworzeniu (pierwsze 30):
['this', 'was', 'an', 'absolutely', 'terrible', 'movie', 'don', 't', 'be', 'lured', 'in', 'by', 'christopher', 'walken', 'or', 'michael', 'ironside', 'both', 'are', 'great', 'actors', 'but', 'this', 'must', 'simply', 'be', 'their', 'worst', 'role', 'in']


In [3]:
# --- IMPORTY I KONFIGURACJA ---
import tensorflow_datasets as tfds
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize

# Pobranie pakietu NLTK 'punkt' do tokenizacji
try:
    nltk.download('punkt')
except:
    pass

# --- ŁADOWANIE DANYCH IMDB REVIEWS ---

(ds_train, ds_test), ds_info = tfds.load(
    'imdb_reviews',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# Konwersja zbiorów TensorFlow na listy Pythona (poprawka błędu z .numpy())
X_train_imdb = [doc.decode('utf8') for doc, label in tfds.as_numpy(ds_train)]
y_train_imdb = [label for doc, label in tfds.as_numpy(ds_train)]

X_test_imdb = [doc.decode('utf8') for doc, label in tfds.as_numpy(ds_test)]
y_test_imdb = [label for doc, label in tfds.as_numpy(ds_test)]

print(f"Dane treningowe: {len(X_train_imdb)} recenzji")
print(f"Dane testowe: {len(X_test_imdb)} recenzji")


# --- FUNKCJA CZYSZCZĄCA I TOKENIZUJĄCA ---

def clean_and_tokenize(text):
    # Usunięcie znaków specjalnych i cyfr
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    # Zmiana na małe litery
    text = text.lower()
    # Usunięcie nadmiarowych białych znaków
    text = re.sub(r'\s+', ' ', text).strip()
    # Tokenizacja
    tokens = word_tokenize(text)
    return tokens


# --- PRZETWARZANIE DANYCH ---

X_train_tokens_imdb = [clean_and_tokenize(doc) for doc in X_train_imdb]
X_test_tokens_imdb = [clean_and_tokenize(doc) for doc in X_test_imdb]


# --- WERYFIKACJA ---
print("\nPrzykład tokenów (pierwsze 30):")
print(X_train_tokens_imdb[0][:30])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rafal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Dane treningowe: 25000 recenzji
Dane testowe: 25000 recenzji

Przykład tokenów (pierwsze 30):
['this', 'was', 'an', 'absolutely', 'terrible', 'movie', 'don', 't', 'be', 'lured', 'in', 'by', 'christopher', 'walken', 'or', 'michael', 'ironside', 'both', 'are', 'great', 'actors', 'but', 'this', 'must', 'simply', 'be', 'their', 'worst', 'role', 'in']
